In [3]:
from parcels import FieldSet, ParticleSet, Variable, JITParticle, ScipyParticle, AdvectionRK4, plotTrajectoriesFile
from parcels import Field
import parcels
import numpy as np
import math
from datetime import timedelta
from operator import attrgetter

##should save import parcels only in the future

### Start with defining fields

In [17]:
class CGS_field:
    """CGS flow field that simulates the environmental flows.
    The field can be initialized by the velocities of flows in given positions.
    """
    
    def __init__(self,x_vel,y_vel):
        """CGS field can be initialized by assigning the velocities in given positions.
        The velocities should be in the form of 2-dimensional numpy arrays, with postive values pointing to right or up.
        For example, if x_vel is [[1,1,1,1],[5,5,5,5]], it means that the velocities in x direction are 1 when y is 1, and 
        are 5 when y is 2. If the velocities are negative, it means the velocities go in negative x direction(to the left).
        """
        
        ## make sure the dimensions of x_vel are equal to those of y_vel
        if x_vel.shape != y_vel.shape:
            raise RuntimeError("The dimensions of x_vel should be the same as those of y_vel")
        
        self.x_velocities = x_vel
        self.y_velocities = y_vel
        
        ## initialize parcels fields with lon and lat values set according to the len(x_vel) and len(y_vel)
        field_x = parcels.Field(name='U',data=x_vel,lon=np.linspace(start=0,stop=1,num=len(x_vel[0])),\
                                lat=np.linspace(start=0,stop=1,num=len(x_vel)))
        field_y = parcels.Field(name='V',data=y_vel,lon=np.linspace(start=0,stop=1,num=len(y_vel[0])),\
                                lat=np.linspace(start=0,stop=1,num=len(y_vel)))
        
        ## initialize parcels fieldsets with fields created above
        fieldset = parcels.FieldSet(U=field_x,V=field_y)
        
        self.fieldset = fieldset
        
    def deploy_kernels(self,x_pos,y_pos):
        """Deploy kernels on the flow field created above. 
        The postions of kernels in x and y directions should be provided in 1-dimension numpy arrays respectively.
        For example, if two kernels are created, and the x_pos and y_pos are set as [1,2] and [3,4], then
        the positions of the kernels will be at [1,3] and [2,4] respectively.
        """
        
        ## the kernels should be customized in the future
        pset = parcels.ParticleSet(fieldset=self.fieldset,pclass=parcels.ScipyParticle,lon=x_pos,lat=y_pos)
        
        self.pset = pset
    
    def simulate(self,time_length,period):
        """Do the simulation.
        The total time length and period of simulation should be provided.
        """
        
        ## do the simulation with time information provided above. outputfile updated with period provided above.
        self.pset.execute(pyfunc=AdvectionRK4,runtime=timedelta(minutes=time_length),dt=timedelta(minutes=period),\
                         outputfile=pset.ParticleFile(name="CGS_simulation", outputdt=timedelta(minutes=period)))
        
    def plot_trajectories(self,file_name,mode):
        """Plot the trajectories of the kernels within the flow field.
        Can choose the moving mode to show the animated trajectories.
        """
        
        if mode == 'moving':
            parcels.plotTrajectoriesFile(filename=file_name, mode='movie2d_notebook')
        else:
            parcels.plotTrajectoriesFile(filename=file_name)